In [1]:
import pandas as pd
pd.options.display.max_columns = 200
pd.options.display.max_rows = 1000
import matplotlib.pyplot as plt
import seaborn as sns
from __future__ import print_function
import os
import subprocess
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier, export_graphviz
%matplotlib inline

In [2]:
sales_pdf = pd.read_csv('../UseCase_3_Datasets/sales_granular.csv')

In [3]:
sales_pdf.shape
sales_pdf.columns

Index(['store_code', '8/3/15 9:00', '8/3/15 10:00', '8/3/15 11:00',
       '8/3/15 12:00', '8/3/15 13:00', '8/3/15 14:00', '8/3/15 15:00',
       '8/3/15 16:00', '8/3/15 17:00',
       ...
       '6/25/17 13:00', '6/25/17 14:00', '6/25/17 15:00', '6/25/17 16:00',
       '6/25/17 17:00', '6/25/17 18:00', '6/25/17 19:00', '6/25/17 20:00',
       '6/25/17 21:00', '6/25/17 22:00'],
      dtype='object', length=11937)

In [4]:
sales_pdf_daily = sales_pdf.fillna(0)

In [5]:
sales_pdf_daily.head()

,store_code,8/3/15 9:00,8/3/15 10:00,8/3/15 11:00,8/3/15 12:00,8/3/15 13:00,8/3/15 14:00,8/3/15 15:00,8/3/15 16:00,8/3/15 17:00,8/3/15 18:00,8/3/15 19:00,8/3/15 22:00,8/4/15 8:00,8/4/15 9:00,8/4/15 10:00,8/4/15 11:00,8/4/15 12:00,8/4/15 13:00,8/4/15 14:00,8/4/15 15:00,8/4/15 16:00,8/4/15 17:00,8/4/15 18:00,8/4/15 19:00,8/4/15 20:00,8/5/15 7:00,8/5/15 9:00,8/5/15 10:00,8/5/15 11:00,8/5/15 12:00,8/5/15 13:00,8/5/15 14:00,8/5/15 15:00,8/5/15 16:00,8/5/15 17:00,8/5/15 18:00,8/5/15 19:00,8/5/15 20:00,8/6/15 9:00,8/6/15 10:00,8/6/15 11:00,8/6/15 12:00,8/6/15 13:00,8/6/15 14:00,8/6/15 15:00,8/6/15 16:00,8/6/15 17:00,8/6/15 18:00,8/6/15 19:00,8/6/15 21:00,8/7/15 6:00,8/7/15 7:00,8/7/15 8:00,8/7/15 9:00,8/7/15 10:00,8/7/15 11:00,8/7/15 12:00,8/7/15 13:00,8/7/15 14:00,8/7/15 15:00,8/7/15 16:00,8/7/15 17:00,8/7/15 18:00,8/7/15 19:00,8/8/15 6:00,8/8/15 7:00,8/8/15 9:00,8/8/15 10:00,8/8/15 11:00,8/8/15 12:00,8/8/15 13:00,8/8/15 14:00,8/8/15 15:00,8/8/15 16:00,8/8/15 17:00,8/8/15 18:00,8/8/15 21:00,8/8/15 22:00,8/9/15 7:00,8/9/15 9:00,8/9/15 11:00,8/9/15 12:00,8/9/15 13:00,8/9/15 14:00,8/9/15 15:00,8/9/15 16:00,8/9/15 18:00,8/9/15 19:00,8/9/15 21:00,8/10/15 7:00,8/10/15 9:00,8/10/15 10:00,8/10/15 11:00,8/10/15 12:00,8/10/15 13:00,8/10/15 14:00,8/10/15 15:00,8/10/15 16:00,8/10/15 17:00,...,6/20/17 17:00,6/20/17 18:00,6/20/17 19:00,6/20/17 20:00,6/20/17 21:00,6/20/17 22:00,6/20/17 23:00,6/21/17 5:00,6/21/17 6:00,6/21/17 7:00,6/21/17 8:00,6/21/17 9:00,6/21/17 10:00,6/21/17 11:00,6/21/17 12:00,6/21/17 13:00,6/21/17 14:00,6/21/17 15:00,6/21/17 16:00,6/21/17 17:00,6/21/17 18:00,6/21/17 19:00,6/21/17 20:00,6/21/17 21:00,6/21/17 22:00,6/22/17 2:00,6/22/17 5:00,6/22/17 6:00,6/22/17 7:00,6/22/17 8:00,6/22/17 9:00,6/22/17 10:00,6/22/17 11:00,6/22/17 12:00,6/22/17 13:00,6/22/17 14:00,6/22/17 15:00,6/22/17 16:00,6/22/17 17:00,6/22/17 18:00,6/22/17 19:00,6/22/17 20:00,6/22/17 21:00,6/22/17 22:00,6/23/17 5:00,6/23/17 6:00,6/23/17 7:00,6/23/17 8:00,6/23/17 9:00,6/23/17 10:00,6/23/17 11:00,6/23/17 12:00,6/23/17 13:00,6/23/17 14:00,6/23/17 15:00,6/23/17 16:00,6/23/17 17:00,6/23/17 18:00,6/23/17 19:00,6/23/17 20:00,6/23/17 21:00,6/23/17 22:00,6/23/17 23:00,6/24/17 0:00,6/24/17 5:00,6/24/17 6:00,6/24/17 7:00,6/24/17 8:00,6/24/17 9:00,6/24/17 10:00,6/24/17 11:00,6/24/17 12:00,6/24/17 13:00,6/24/17 14:00,6/24/17 15:00,6/24/17 16:00,6/24/17 17:00,6/24/17 18:00,6/24/17 19:00,6/24/17 20:00,6/24/17 21:00,6/24/17 22:00,6/24/17 23:00,6/25/17 6:00,6/25/17 7:00,6/25/17 8:00,6/25/17 9:00,6/25/17 10:00,6/25/17 11:00,6/25/17 12:00,6/25/17 13:00,6/25/17 14:00,6/25/17 15:00,6/25/17 16:00,6/25/17 17:00,6/25/17 18:00,6/25/17 19:00,6/25/17 20:00,6/25/17 21:00,6/25/17 22:00
0,10055,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,30.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10077,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [6]:
#Calculate the no.of sales per hour, in the above sales granular information.
#Filter , columsn where a lot of entry is empty .
from datetime import datetime as dt

col_list = sales_pdf_daily.columns
for col in col_list[1:]:
    if dt.strptime(col.split(" ")[0],"%m/%d/%y") < dt.strptime("10/1/16", "%m/%d/%y"):
        #print ("Cols to delete", col)
        sales_pdf_daily.drop(col, axis = 1, inplace = True)

In [7]:
sales_pdf_daily.head()

,store_code,10/1/16 6:00,10/1/16 7:00,10/1/16 8:00,10/1/16 9:00,10/1/16 10:00,10/1/16 11:00,10/1/16 12:00,10/1/16 13:00,10/1/16 14:00,10/1/16 15:00,10/1/16 16:00,10/1/16 17:00,10/1/16 18:00,10/1/16 19:00,10/1/16 20:00,10/1/16 21:00,10/1/16 22:00,10/1/16 23:00,10/2/16 5:00,10/2/16 6:00,10/2/16 7:00,10/2/16 8:00,10/2/16 9:00,10/2/16 10:00,10/2/16 11:00,10/2/16 12:00,10/2/16 13:00,10/2/16 14:00,10/2/16 15:00,10/2/16 16:00,10/2/16 17:00,10/2/16 18:00,10/2/16 19:00,10/2/16 20:00,10/2/16 21:00,10/2/16 22:00,10/2/16 23:00,10/3/16 6:00,10/3/16 7:00,10/3/16 8:00,10/3/16 9:00,10/3/16 10:00,10/3/16 11:00,10/3/16 12:00,10/3/16 13:00,10/3/16 14:00,10/3/16 15:00,10/3/16 16:00,10/3/16 17:00,10/3/16 18:00,10/3/16 19:00,10/3/16 20:00,10/3/16 21:00,10/3/16 22:00,10/4/16 5:00,10/4/16 6:00,10/4/16 7:00,10/4/16 8:00,10/4/16 9:00,10/4/16 10:00,10/4/16 11:00,10/4/16 12:00,10/4/16 13:00,10/4/16 14:00,10/4/16 15:00,10/4/16 16:00,10/4/16 17:00,10/4/16 18:00,10/4/16 19:00,10/4/16 20:00,10/4/16 21:00,10/4/16 22:00,10/5/16 5:00,10/5/16 6:00,10/5/16 7:00,10/5/16 8:00,10/5/16 9:00,10/5/16 10:00,10/5/16 11:00,10/5/16 12:00,10/5/16 13:00,10/5/16 14:00,10/5/16 15:00,10/5/16 16:00,10/5/16 17:00,10/5/16 18:00,10/5/16 19:00,10/5/16 20:00,10/5/16 21:00,10/5/16 23:00,10/6/16 5:00,10/6/16 6:00,10/6/16 7:00,10/6/16 8:00,10/6/16 9:00,10/6/16 10:00,10/6/16 11:00,10/6/16 12:00,10/6/16 13:00,...,6/20/17 17:00,6/20/17 18:00,6/20/17 19:00,6/20/17 20:00,6/20/17 21:00,6/20/17 22:00,6/20/17 23:00,6/21/17 5:00,6/21/17 6:00,6/21/17 7:00,6/21/17 8:00,6/21/17 9:00,6/21/17 10:00,6/21/17 11:00,6/21/17 12:00,6/21/17 13:00,6/21/17 14:00,6/21/17 15:00,6/21/17 16:00,6/21/17 17:00,6/21/17 18:00,6/21/17 19:00,6/21/17 20:00,6/21/17 21:00,6/21/17 22:00,6/22/17 2:00,6/22/17 5:00,6/22/17 6:00,6/22/17 7:00,6/22/17 8:00,6/22/17 9:00,6/22/17 10:00,6/22/17 11:00,6/22/17 12:00,6/22/17 13:00,6/22/17 14:00,6/22/17 15:00,6/22/17 16:00,6/22/17 17:00,6/22/17 18:00,6/22/17 19:00,6/22/17 20:00,6/22/17 21:00,6/22/17 22:00,6/23/17 5:00,6/23/17 6:00,6/23/17 7:00,6/23/17 8:00,6/23/17 9:00,6/23/17 10:00,6/23/17 11:00,6/23/17 12:00,6/23/17 13:00,6/23/17 14:00,6/23/17 15:00,6/23/17 16:00,6/23/17 17:00,6/23/17 18:00,6/23/17 19:00,6/23/17 20:00,6/23/17 21:00,6/23/17 22:00,6/23/17 23:00,6/24/17 0:00,6/24/17 5:00,6/24/17 6:00,6/24/17 7:00,6/24/17 8:00,6/24/17 9:00,6/24/17 10:00,6/24/17 11:00,6/24/17 12:00,6/24/17 13:00,6/24/17 14:00,6/24/17 15:00,6/24/17 16:00,6/24/17 17:00,6/24/17 18:00,6/24/17 19:00,6/24/17 20:00,6/24/17 21:00,6/24/17 22:00,6/24/17 23:00,6/25/17 6:00,6/25/17 7:00,6/25/17 8:00,6/25/17 9:00,6/25/17 10:00,6/25/17 11:00,6/25/17 12:00,6/25/17 13:00,6/25/17 14:00,6/25/17 15:00,6/25/17 16:00,6/25/17 17:00,6/25/17 18:00,6/25/17 19:00,6/25/17 20:00,6/25/17 21:00,6/25/17 22:00
0,10055,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,30.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10077,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [9]:
sales_pdf_daily['total_sales'] = sales_pdf_daily.sum(axis=1)
sales_pdf_daily['sales_per_hr'] = sales_pdf_daily['total_sales']/sales_pdf_daily.shape[1]
sales_pdf_daily['sales_per_hr'].describe()

count    906.000000
mean      28.920646
std       51.693408
min        0.501778
25%       11.691823
50%       14.739868
75%       25.374810
max      558.394318
Name: sales_per_hr, dtype: float64

In [10]:
sales_pdf_daily['sales_per_hr'].hist()
